# IU Applied Data Science (INFO_I-590)
### Instructor: Joanne Luciano ; TAs: Jeremy Yang and Kaicheng Yang
# ADS Twitter API Tutorial
### Based on original tutorial and code by former TA JT Wolohan.
https://media.readthedocs.org/pdf/twitterapi/latest/twitterapi.pdf

In [0]:
import os, json
import pandas
import numpy
import sklearn
import sklearn.feature_extraction
import sklearn.model_selection 
import sklearn.metrics 
import sklearn.naive_bayes
import sklearn.svm
import sklearn.neighbors
import sklearn.neural_network
!pip install TwitterAPI
from TwitterAPI import TwitterAPI, TwitterOAuth

### Twitter API Credentials
### (IN ACCORDANCE WITH TWITTER LICENSE AGGREEMENT, DO NOT COPY, USE ONLY YOUR OWN CREDENTIALS.)

In [0]:
consumer_key_b64 = 'cmJqbG1US1JwdU04ZXhJQlBPWnlvbFZRSw=='
consumer_secret_b64  = 'enNrMjdSNGlLbElOUzN4YWk3WHFzQnAxVkdyaU1xU0FFYzlDSGd5SUhselZvRjZ1dmw='
access_token_key_b64 = 'NjY4MzUxMDMtN3dKUFpPUHhjZERzbkY0RkViQ0E3RlFnYjhENEFQYjk1TDhaaWZrZUo='
access_token_secret_b64 = 'S0dDczUyZnZvRjhRRDZPcktEU3JxRlFhSWZOZkRnTng2aWJlWUVrZWt0Y3F0'

### SAVE YOUR APPLICATION CREDENTIALS
Credentials file format:

```
consumer_key=YOUR_CONSUMER_KEY
consumer_secret=YOUR_CONSUMER_SECRET
access_token_key=YOUR_ACCESS_TOKEN
access_token_secret=YOUR_ACCESS_TOKEN_SECRET
```

In [0]:
from base64 import b64decode

consumer_key = str(b64decode(consumer_key_b64), 'utf8')
consumer_secret = str(b64decode(consumer_secret_b64), 'utf8')
access_token_key = str(b64decode(access_token_key_b64), 'utf8')
access_token_secret = str(b64decode(access_token_secret_b64), 'utf8')

fpath = os.environ['HOME']+'/.twitterapi_credentials'
fout = open(fpath, "w")
fout.write("consumer_key=%s\n"%consumer_key)
fout.write("consumer_secret=%s\n"%consumer_secret)
fout.write("access_token_key=%s\n"%access_token_key)
fout.write("access_token_secret=%s\n"%access_token_secret)
fout.close()

### Connect and authenticate with Twitter REST API.

In [0]:
o = TwitterOAuth.read_file(os.environ['HOME']+'/.twitterapi_credentials')
twapi = TwitterAPI(o.consumer_key, o.consumer_secret, o.access_token_key, o.access_token_secret)

### Convenience functions

In [0]:
def print_score(Ytrue,Ypred):
  s = (sklearn.metrics.precision_score(Ytrue,Ypred),
          sklearn.metrics.recall_score(Ytrue,Ypred),
          sklearn.metrics.f1_score(Ytrue,Ypred))
  print('Precision: {:0.3}\nRecall: {:0.3}\nF-Score: {:0.3}\n'.format(*s))


def searchTwitter(q,api,feed="search/tweets",n=100):
  return [t for t in api.request(feed, {'q':q,'count':n})]

### Get JSON from Twitter

In [0]:
cat_tweets = searchTwitter('#cats', twapi, n=100)
dog_tweets = searchTwitter('#dogs', twapi, n=100)

### Convert the json returned by Twitter into a dataframe

In [0]:
cat_df = pandas.read_json(json.dumps(cat_tweets))
dog_df = pandas.read_json(json.dumps(dog_tweets))

### Inspect data frames.


In [0]:
cat_df.head()
dog_df.head()

### Get text only and replace hashtags with blanks
If you want to use the normalizer, import it above and pass x.replace() to the noramlizer function

In [0]:
cat_txt = [x.replace('#cats',"") for x in cat_df['text']]
dog_txt = [x.replace('#dogs',"") for x in dog_df['text']]

### Create features and return sparse matricies

In [0]:
vectorizer = sklearn.feature_extraction.text.CountVectorizer(cat_txt+dog_txt)
vectorizer.fit(cat_txt+dog_txt)
cat_tdm = vectorizer.transform(cat_txt).toarray()
dog_tdm = vectorizer.transform(dog_txt).toarray()

### Create visible matricies and combine

In [0]:
zeros = numpy.zeros((len(cat_txt),1))
ones = numpy.ones((len(dog_txt),1))
catdog_tdm = numpy.concatenate((cat_tdm,dog_tdm),axis=0)
Y = numpy.ravel(numpy.concatenate((zeros,ones),axis=0))

### Create train/test split for modeling

In [0]:
Xtrain,Xtest,Ytrain,Ytest = sklearn.model_selection.train_test_split(catdog_tdm, Y, test_size=.25)

### Naive Bayes

In [0]:
nb = sklearn.naive_bayes.GaussianNB()
nb.fit(Xtrain,Ytrain)
Ypred = nb.predict(Xtest)

print("\nNaive Bayes Performance")
print_score(Ytest,Ypred)

### SVM

In [0]:
svm = sklearn.svm.SVC()
svm.fit(Xtrain,Ytrain)
Ypred = svm.predict(Xtest)

print("\nSVM performance")
print_score(Ytest,Ypred)

### Neural Network

In [0]:
nn = sklearn.neural_network.MLPClassifier()
nn.fit(Xtrain,Ytrain)
Ypred = nn.predict(Xtest)

print("\nNeural Network Performance")
print_score(Ytest,Ypred)

### KNN

In [0]:
knn1 = sklearn.neighbors.KNeighborsClassifier(n_neighbors=1)
knn5 = sklearn.neighbors.KNeighborsClassifier(n_neighbors=5)
knn5d = sklearn.neighbors.KNeighborsClassifier(n_neighbors=5,weights='distance')

knn1.fit(Xtrain,Ytrain)
knn5.fit(Xtrain,Ytrain)
knn5d.fit(Xtrain,Ytrain)

print("\nKNN 1 Neighbor Performance")
print_score(Ytest,knn1.predict(Xtest))

print("\nKNN 5 Neighbor Performance")
print_score(Ytest,knn5.predict(Xtest))

print("\nKNN 5 Neighbor Weighted Performance")
print_score(Ytest,knn5d.predict(Xtest))